In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

start running the code

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications import Xception, InceptionV3, VGG16, ResNet50
from tensorflow.keras.applications import MobileNetV2, EfficientNetB0, DenseNet121
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score
import os # Importing the os module

# Specifying train, valid and test directory paths
# Use os.path.join to create correct paths
train_path = os.path.join('/content/drive/MyDrive/FYP/dataset', 'train')
valid_path = os.path.join('/content/drive/MyDrive/FYP/dataset', 'validation')
test_path = os.path.join('/content/drive/MyDrive/FYP/dataset', 'test')

# Rescaling the image
train_batches = ImageDataGenerator(rescale=1.0 / 255.0)
valid_batches = ImageDataGenerator(rescale=1.0 / 255.0)
test_batches = ImageDataGenerator(rescale=1.0 / 255.0)

In [ ]:
# Train data generator
train_gen = train_batches.flow_from_directory(
    directory=train_path,
    target_size=(224, 224),
    classes=['angry', 'happy', 'neutral', 'sad'],
    batch_size=32
)

# Valid data generator
valid_gen = valid_batches.flow_from_directory(
    directory=valid_path,
    target_size=(224, 224),
    classes=['angry', 'happy', 'neutral', 'sad'],
    batch_size=32
)

# Test data generator
test_gen = test_batches.flow_from_directory(
    directory=test_path,
    target_size=(224, 224),
    classes=['angry', 'happy', 'neutral', 'sad'],
    batch_size=32,
    shuffle=False
)

Found 1764 images belonging to 4 classes.
Found 376 images belonging to 4 classes.
Found 380 images belonging to 4 classes.


INITIALIZING MODELS



In [ ]:
# Initializing models
model_vgg = VGG16(include_top=False, input_shape=(224, 224, 3))
model_xception = Xception(include_top=False, input_shape=(224, 224, 3))
model_inception = InceptionV3(include_top=False, input_shape=(224, 224, 3))
model_resnet = ResNet50(include_top=False, input_shape=(224, 224, 3))
model_mobilenet = MobileNetV2(include_top=False, input_shape=(224, 224, 3))
model_efficientnet = EfficientNetB0(include_top=False, input_shape=(224, 224, 3))
model_densenet = DenseNet121(include_top=False, input_shape=(224, 224, 3))

In [ ]:
# Freezing the layers so that they cannot be trained again
names = [model_vgg, model_xception, model_inception, model_resnet, model_mobilenet, model_efficientnet, model_densenet]
for model in names:
    # Iterating all the layers in the pre-trained model
    for layer in model.layers:
        # Making trainable layers set to False
        layer.trainable = False

In [ ]:
# Fine-tuning the pre-trained models
output_classes = len(train_gen.class_indices)

# Custom VGG16 model
custom_vgg_model = Sequential([
    model_vgg,
    GlobalAveragePooling2D(),  # Faster than Flatten()
    Dense(256, activation='relu'),
    Dropout(0.3),  # Reduce dropout for speed
    Dense(output_classes, activation='softmax')
])

# Custom Xception model
custom_xc_model = Sequential([
    model_xception,
    GlobalAveragePooling2D(),
    Dense(output_classes, activation='softmax')
])

# Custom Inception model
custom_inc_model = Sequential([
    model_inception,
    GlobalAveragePooling2D(),
    Dense(output_classes, activation='softmax')
])

# Custom ResNet model
custom_resnet_model = Sequential([
    model_resnet,
    GlobalAveragePooling2D(),
    Dense(output_classes, activation='softmax')
])

# Custom MobileNetV2 model
custom_mobilenet_model = Sequential([
    model_mobilenet,
    GlobalAveragePooling2D(),
    Dense(output_classes, activation='softmax')
])

# Custom EfficientNetB0 model
custom_efficientnet_model = Sequential([
    model_efficientnet,
    GlobalAveragePooling2D(),
    Dense(output_classes, activation='softmax')
])

# Custom DenseNet121 model
custom_densenet_model = Sequential([
    model_densenet,
    GlobalAveragePooling2D(),
    Dense(output_classes, activation='softmax')
])

In [ ]:
# Compiling the model
models = [custom_vgg_model, custom_xc_model, custom_inc_model, custom_resnet_model]

for model in models:
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

In [ ]:
# Compiling the models
models = [custom_mobilenet_model, custom_efficientnet_model, custom_densenet_model]

for model in models:
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

VGG16

In [ ]:
import time
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_histories = []

print(">>> Training VGG16 model:")
start_time = time.time()

result_vgg = custom_vgg_model.fit(
    train_gen,
    validation_data=valid_gen,
    epochs=100,
    callbacks=[early_stopping],
    verbose=1
)

end_time = time.time()
elapsed_time = end_time - start_time
epochs_completed = len(result_vgg.history['loss'])

model_histories.append((result_vgg.history, "VGG16"))

print(f">>> Training completed in {elapsed_time:.2f} seconds.")
print(f">>> Epochs completed: {epochs_completed}")

print(">>> Evaluating VGG16 on the Test data:")
test_pred_vgg = custom_vgg_model.predict(test_gen)
test_labels = test_gen.classes
test_accuracy_vgg = accuracy_score(np.argmax(test_pred_vgg, axis=1), test_labels)
print(f">>> Test Accuracy for VGG16: {test_accuracy_vgg * 100:.2f}%.")

>>> Training VGG16 model:
Epoch 1/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 13s 238ms/step - accuracy: 0.9840 - loss: 0.1033 - val_accuracy: 0.9628 - val_loss: 0.1373
Epoch 2/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 13s 224ms/step - accuracy: 0.9821 - loss: 0.1040 - val_accuracy: 0.9521 - val_loss: 0.1564
Epoch 3/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 12s 212ms/step - accuracy: 0.9663 - loss: 0.1311 - val_accuracy: 0.9734 - val_loss: 0.1249
Epoch 4/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 12s 208ms/step - accuracy: 0.9809 - loss: 0.0936 - val_accuracy: 0.9441 - val_loss: 0.1752
Epoch 5/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 12s 217ms/step - accuracy: 0.9716 - loss: 0.1078 - val_accuracy: 0.9681 - val_loss: 0.1225
Epoch 6/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 20s 209ms/step - accuracy: 0.9930 - loss: 0.0718 - val_accuracy: 0.9814 - val_loss: 0.1121
Epoch 7/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 12s 212ms/step - accuracy: 0.9904 - loss: 0.0879 - val_accuracy: 0.9681 - val_loss: 0.1309
Epoch 8/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 12s 220ms/step - accuracy

Xception

In [ ]:
import time
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

print(">>> Training Xception model:")
start_time = time.time()

result_xc = custom_xc_model.fit(
    train_gen,
    validation_data=valid_gen,
    epochs=100,
    callbacks=[early_stopping],
    verbose=1
)

end_time = time.time()
elapsed_time = end_time - start_time
epochs_completed = len(result_xc.history['loss'])

model_histories.append((result_xc.history, "Xception"))

print(f">>> Training completed in {elapsed_time:.2f} seconds.")
print(f">>> Epochs completed: {epochs_completed}")

print(">>> Evaluating Xception on the Test data:")
test_pred_xc = custom_xc_model.predict(test_gen)
test_labels = test_gen.classes
test_accuracy_xc = accuracy_score(np.argmax(test_pred_xc, axis=1), test_labels)
print(f">>> Test Accuracy for Xception: {test_accuracy_xc * 100:.2f}%.")

>>> Training Xception model:
Epoch 1/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 36s 412ms/step - accuracy: 0.2821 - loss: 1.4581 - val_accuracy: 0.3537 - val_loss: 1.2980
Epoch 2/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 10s 184ms/step - accuracy: 0.4634 - loss: 1.2433 - val_accuracy: 0.4521 - val_loss: 1.2147
Epoch 3/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 21s 188ms/step - accuracy: 0.5166 - loss: 1.1512 - val_accuracy: 0.5878 - val_loss: 1.0896
Epoch 4/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 11s 189ms/step - accuracy: 0.6056 - loss: 1.0649 - val_accuracy: 0.6543 - val_loss: 1.0392
Epoch 5/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 10s 180ms/step - accuracy: 0.6797 - loss: 0.9911 - val_accuracy: 0.6888 - val_loss: 0.9721
Epoch 6/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 10s 177ms/step - accuracy: 0.7584 - loss: 0.9079 - val_accuracy: 0.6862 - val_loss: 0.9350
Epoch 7/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 11s 189ms/step - accuracy: 0.7545 - loss: 0.8586 - val_accuracy: 0.7101 - val_loss: 0.8903
Epoch 8/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 11s 186ms/step - accur

InceptionV3

In [ ]:
import time
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

print(">>> Training InceptionV3 model:")
start_time = time.time()

result_inc = custom_inc_model.fit(
    train_gen,
    validation_data=valid_gen,
    epochs=100,
    callbacks=[early_stopping],
    verbose=1
)

end_time = time.time()
elapsed_time = end_time - start_time
epochs_completed = len(result_inc.history['loss'])

model_histories.append((result_inc.history, "InceptionV3"))

print(f">>> Training completed in {elapsed_time:.2f} seconds.")
print(f">>> Epochs completed: {epochs_completed}")

print(">>> Evaluating InceptionV3 on the Test data:")
test_pred_inc = custom_inc_model.predict(test_gen)
test_labels = test_gen.classes
test_accuracy_inc = accuracy_score(np.argmax(test_pred_inc, axis=1), test_labels)
print(f">>> Test Accuracy for InceptionV3: {test_accuracy_inc * 100:.2f}%.")

>>> Training InceptionV3 model:
Epoch 1/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 60s 792ms/step - accuracy: 0.2675 - loss: 1.6553 - val_accuracy: 0.3245 - val_loss: 1.3207
Epoch 2/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 10s 179ms/step - accuracy: 0.4810 - loss: 1.1809 - val_accuracy: 0.5532 - val_loss: 1.1235
Epoch 3/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 10s 178ms/step - accuracy: 0.6125 - loss: 1.0106 - val_accuracy: 0.6011 - val_loss: 1.0131
Epoch 4/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 10s 185ms/step - accuracy: 0.7050 - loss: 0.8818 - val_accuracy: 0.6064 - val_loss: 0.9639
Epoch 5/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 10s 172ms/step - accuracy: 0.7473 - loss: 0.7764 - val_accuracy: 0.6729 - val_loss: 0.8455
Epoch 6/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 10s 178ms/step - accuracy: 0.7990 - loss: 0.6987 - val_accuracy: 0.7766 - val_loss: 0.7606
Epoch 7/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 11s 189ms/step - accuracy: 0.8539 - loss: 0.6192 - val_accuracy: 0.7926 - val_loss: 0.7095
Epoch 8/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 9s 163ms/step - acc

ResNet50

In [ ]:
import time
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

print(">>> Training ResNet50 model:")
start_time = time.time()

result_resnet = custom_resnet_model.fit(
    train_gen,
    validation_data=valid_gen,
    epochs=100,
    callbacks=[early_stopping],
    verbose=1
)

end_time = time.time()
elapsed_time = end_time - start_time
epochs_completed = len(result_resnet.history['loss'])

model_histories.append((result_resnet.history, "ResNet50"))

print(f">>> Training completed in {elapsed_time:.2f} seconds.")
print(f">>> Epochs completed: {epochs_completed}")

print(">>> Evaluating ResNet50 on the Test data:")
test_pred_resnet = custom_resnet_model.predict(test_gen)
test_labels = test_gen.classes
test_accuracy_resnet = accuracy_score(np.argmax(test_pred_resnet, axis=1), test_labels)
print(f">>> Test Accuracy for ResNet50: {test_accuracy_resnet * 100:.2f}%.")

>>> Training ResNet50 model:
Epoch 1/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 34s 417ms/step - accuracy: 0.2646 - loss: 1.4023 - val_accuracy: 0.2553 - val_loss: 1.4477
Epoch 2/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 10s 184ms/step - accuracy: 0.2485 - loss: 1.4099 - val_accuracy: 0.2606 - val_loss: 1.4250
Epoch 3/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 10s 184ms/step - accuracy: 0.2546 - loss: 1.4003 - val_accuracy: 0.2819 - val_loss: 1.3715
Epoch 4/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 20s 180ms/step - accuracy: 0.2781 - loss: 1.3750 - val_accuracy: 0.2500 - val_loss: 1.3921
Epoch 5/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 10s 184ms/step - accuracy: 0.2543 - loss: 1.3815 - val_accuracy: 0.2660 - val_loss: 1.3835
Epoch 6/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 10s 173ms/step - accuracy: 0.2987 - loss: 1.3683 - val_accuracy: 0.3883 - val_loss: 1.3916
Epoch 7/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 10s 174ms/step - accuracy: 0.3171 - loss: 1.3628 - val_accuracy: 0.3191 - val_loss: 1.3747
Epoch 8/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 10s 179ms/step - accur

MobileNetV2

In [ ]:
import time
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

print(">>> Training MobileNetV2 model:")
start_time = time.time()

result_mob = custom_mobilenet_model.fit(
    train_gen,
    validation_data=valid_gen,
    epochs=100,
    callbacks=[early_stopping],
    verbose=1
)

end_time = time.time()
elapsed_time = end_time - start_time
epochs_completed = len(result_mob.history['loss'])

model_histories.append((result_mob.history, "MobileNetV2"))

print(f">>> Training completed in {elapsed_time:.2f} seconds.")
print(f">>> Epochs completed: {epochs_completed}")

print(">>> Evaluating MobileNetV2 on the Test data:")
test_pred_mob = custom_mobilenet_model.predict(test_gen)
test_labels = test_gen.classes
test_accuracy_mob = accuracy_score(np.argmax(test_pred_mob, axis=1), test_labels)
print(f">>> Test Accuracy for MobileNetV2: {test_accuracy_mob * 100:.2f}%.")


>>> Training MobileNetV2 model:
Epoch 1/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 30s 365ms/step - accuracy: 0.2471 - loss: 1.5610 - val_accuracy: 0.3378 - val_loss: 1.3512
Epoch 2/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 10s 173ms/step - accuracy: 0.3975 - loss: 1.2878 - val_accuracy: 0.5612 - val_loss: 1.1374
Epoch 3/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 9s 162ms/step - accuracy: 0.5648 - loss: 1.1014 - val_accuracy: 0.6170 - val_loss: 1.0370
Epoch 4/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 10s 170ms/step - accuracy: 0.6608 - loss: 0.9905 - val_accuracy: 0.6596 - val_loss: 0.9704
Epoch 5/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 10s 170ms/step - accuracy: 0.7129 - loss: 0.8966 - val_accuracy: 0.7553 - val_loss: 0.8721
Epoch 6/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 9s 160ms/step - accuracy: 0.7677 - loss: 0.8166 - val_accuracy: 0.7553 - val_loss: 0.8303
Epoch 7/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 9s 169ms/step - accuracy: 0.7860 - loss: 0.7635 - val_accuracy: 0.7899 - val_loss: 0.7479
Epoch 8/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 10s 173ms/step - accur

EfficientNetB0

In [ ]:
import time
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

print(">>> Training EfficientNetB0 model:")
start_time = time.time()

result_eff = custom_efficientnet_model.fit(
    train_gen,
    validation_data=valid_gen,
    epochs=100,
    callbacks=[early_stopping],
    verbose=1
)

end_time = time.time()
elapsed_time = end_time - start_time
epochs_completed = len(result_eff.history['loss'])

model_histories.append((result_eff.history, "EfficientNetB0"))

print(f">>> Training completed in {elapsed_time:.2f} seconds.")
print(f">>> Epochs completed: {epochs_completed}")

print(">>> Evaluating EfficientNetB0 on the Test data:")
test_pred_eff = custom_efficientnet_model.predict(test_gen)
test_labels = test_gen.classes
test_accuracy_eff = accuracy_score(np.argmax(test_pred_eff, axis=1), test_labels)
print(f">>> Test Accuracy for EfficientNetB0: {test_accuracy_eff * 100:.2f}%.")


>>> Training EfficientNetB0 model:
Epoch 1/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 10s 180ms/step - accuracy: 0.2896 - loss: 1.4018 - val_accuracy: 0.2500 - val_loss: 1.3964
Epoch 2/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 12s 216ms/step - accuracy: 0.2597 - loss: 1.3942 - val_accuracy: 0.2500 - val_loss: 1.3887
Epoch 3/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 10s 170ms/step - accuracy: 0.2695 - loss: 1.4001 - val_accuracy: 0.2500 - val_loss: 1.4025
Epoch 4/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 10s 164ms/step - accuracy: 0.2349 - loss: 1.4041 - val_accuracy: 0.2500 - val_loss: 1.3881
Epoch 5/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 10s 177ms/step - accuracy: 0.2560 - loss: 1.3996 - val_accuracy: 0.2500 - val_loss: 1.4334
Epoch 6/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 10s 171ms/step - accuracy: 0.2466 - loss: 1.4100 - val_accuracy: 0.2500 - val_loss: 1.3952
Epoch 7/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 9s 161ms/step - accuracy: 0.2409 - loss: 1.4045 - val_accuracy: 0.2500 - val_loss: 1.3878
Epoch 8/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 9s 168ms/step - a

DenseNet121

In [ ]:
import time
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

print(">>> Training DenseNet121 model:")
start_time = time.time()

result_densenet = custom_densenet_model.fit(
    train_gen,
    validation_data=valid_gen,
    epochs=100,
    callbacks=[early_stopping],
    verbose=1
)

end_time = time.time()
elapsed_time = end_time - start_time
epochs_completed = len(result_densenet.history['loss'])

model_histories.append((result_densenet.history, "DenseNet121"))

print(f">>> Training completed in {elapsed_time:.2f} seconds.")
print(f">>> Epochs completed: {epochs_completed}")

print(">>> Evaluating DenseNet121 on the Test data:")
test_pred_densenet = custom_densenet_model.predict(test_gen)
test_labels = test_gen.classes
test_accuracy_densenet = accuracy_score(np.argmax(test_pred_densenet, axis=1), test_labels)
print(f">>> Test Accuracy for DenseNet121: {test_accuracy_densenet * 100:.2f}%.")


>>> Training DenseNet121 model:
Epoch 1/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 92s 938ms/step - accuracy: 0.2699 - loss: 1.5129 - val_accuracy: 0.3112 - val_loss: 1.4038
Epoch 2/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 11s 194ms/step - accuracy: 0.3191 - loss: 1.3670 - val_accuracy: 0.3910 - val_loss: 1.3196
Epoch 3/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 10s 170ms/step - accuracy: 0.4149 - loss: 1.2790 - val_accuracy: 0.4548 - val_loss: 1.2620
Epoch 4/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 10s 184ms/step - accuracy: 0.5169 - loss: 1.1910 - val_accuracy: 0.4548 - val_loss: 1.2252
Epoch 5/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 11s 194ms/step - accuracy: 0.5526 - loss: 1.1464 - val_accuracy: 0.5053 - val_loss: 1.1737
Epoch 6/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 11s 190ms/step - accuracy: 0.5932 - loss: 1.0977 - val_accuracy: 0.5957 - val_loss: 1.1060
Epoch 7/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 10s 172ms/step - accuracy: 0.6499 - loss: 1.0404 - val_accuracy: 0.6170 - val_loss: 1.0846
Epoch 8/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 11s 188ms/step - ac

FOCUSED: ONLY DENSENET121  
AFTER ENHANCEMENT

In [ ]:
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout, BatchNormalization, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score
import numpy as np

model_densenet = DenseNet121(include_top=False, input_shape=(224, 224, 3))

# Define the DenseNet121 model with a custom top
base_model = DenseNet121(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))
base_model.trainable = False  # Freeze base for initial training

# >>> Custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)

x = Dense(512)(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dropout(0.5)(x)

x = Dense(256)(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dropout(0.4)(x)

output = Dense(4, activation='softmax')(x)  # Final output for 4 emotion classes

# Final model
custom_densenet_model = Model(inputs=base_model.input, outputs=output)

# Compile
custom_densenet_model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the DenseNet121 model
print(">>> Training DenseNet121 model:")

result = custom_densenet_model.fit(
    train_gen,
    validation_data=valid_gen,
    epochs=100,
    callbacks=[early_stopping],
    verbose=1
)

# Store the training history
model_histories = [(result.history, "DenseNet121")]

# Evaluate on the test data
print(">>> Evaluating DenseNet121 on the Test data:")
test_pred = custom_densenet_model.predict(test_gen)
test_labels = test_gen.classes
test_accuracy = accuracy_score(np.argmax(test_pred, axis=1), test_labels)
print(f">>> Test Accuracy for DenseNet121: {test_accuracy * 100:.2f}%.")

>>> Training DenseNet121 model:


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_2155']
Received: inputs=Tensor(shape=(None, 224, 224, 3))
  warnings.warn(msg)


56/56 ━━━━━━━━━━━━━━━━━━━━ 74s 798ms/step - accuracy: 0.2721 - loss: 1.8598 - val_accuracy: 0.3298 - val_loss: 1.3986
Epoch 2/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 14s 243ms/step - accuracy: 0.3959 - loss: 1.5423 - val_accuracy: 0.4202 - val_loss: 1.2661
Epoch 3/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 13s 224ms/step - accuracy: 0.3869 - loss: 1.4812 - val_accuracy: 0.5479 - val_loss: 1.1444
Epoch 4/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 13s 224ms/step - accuracy: 0.4326 - loss: 1.2957 - val_accuracy: 0.6330 - val_loss: 1.0271
Epoch 5/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 13s 235ms/step - accuracy: 0.4587 - loss: 1.2594 - val_accuracy: 0.6782 - val_loss: 0.9337
Epoch 6/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 20s 228ms/step - accuracy: 0.5129 - loss: 1.1409 - val_accuracy: 0.7340 - val_loss: 0.8392
Epoch 7/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 13s 224ms/step - accuracy: 0.5394 - loss: 1.1242 - val_accuracy: 0.7872 - val_loss: 0.7611
Epoch 8/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 12s 218ms/step - accuracy: 0.5987 - loss: 0.9893 - val_accuracy

/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_2155']
Received: inputs=Tensor(shape=(32, 224, 224, 3))
  warnings.warn(msg)


12/12 ━━━━━━━━━━━━━━━━━━━━ 27s 2s/step
>>> Test Accuracy for DenseNet121: 98.42%.
